# Model Monitoring KPI Example
This notebook demonstrates how to use the `model_monitoring` module to define and analyze Key Performance Indicators (KPIs) across different data segments.

In [2]:
# Install missing dependencies
%pip install pyarrow scipy fastparquet matplotlib seaborn plotly

# Workaround for PyArrow extension type conflict
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pyarrow')

# Clear any existing PyArrow extension types to avoid conflicts
try:
    import pyarrow as pa
    # This helps avoid the extension type registration conflict
    import importlib
    if hasattr(pa, '_extension_types'):
        pa._extension_types.clear()
except:
    pass

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from model_monitoring import (
    AnalysisDataBuilder,
    SegmentCategorical,
    SegmentCustom,
    calculate_statistics,
)
from model_monitoring.plotting import plot_segment_statistics, set_plot_theme

print("✓ All imports successful! model_monitoring package is now available.")

  Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.59.0-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (107 kB)
  Using cached fonttools-4.59.0-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (107 kB)
  Using ca

## 1. Generate Synthetic Data

First, we'll create a synthetic dataset. This dataset mimics a typical insurance scenario with multiple coverages, predictions (risk premiums), and observed outcomes (targets/claims). The data is saved to a parquet file.

In [3]:
# --- Create a Synthetic Dataset ---
np.random.seed(42)
n_samples = 10000

# Create some numerical features
features = pd.DataFrame(
    {
        "age": np.random.randint(18, 70, n_samples),
        "income": np.random.gamma(2, 40000, n_samples),
        "credit_score": np.random.randint(300, 850, n_samples),
        "market_premium": np.random.uniform(0.1, 0.5, n_samples),
    }
)

# Create a categorical feature
features["region"] = np.random.choice(
    ["North", "South", "East", "West"], n_samples, p=[0.3, 0.2, 0.25, 0.25]
)

# --- Generate data for N=3 coverages in a wide format ---
N_coverages = 3
df = features.copy()

for i in range(N_coverages):
    cov_suffix = f"_{chr(65 + i)}"  # e.g., _A, _B, _C

    # --- Generate True Risk (rate for Poisson) ---
    true_risk_formula = (
        -4.0
        + (i * 0.1)
        + df["age"] / (20 + i * 2)
        - df["credit_score"] / (500 + i * 20)
        + df["income"] / 100000
    )
    true_risk_index = np.exp(true_risk_formula)

    # --- Generate a Slightly Incorrect Prediction ---
    prediction_formula = (
        -3.9
        + (i * 0.1)
        + df["age"] / (22 + i * 2)
        - df["credit_score"] / (550 + i * 20)
        + df["income"] / 110000
    )
    df[f"prediction{cov_suffix}"] = np.exp(prediction_formula)

    # --- Generate Target (claims) from the true risk ---
    df[f"target{cov_suffix}"] = np.random.poisson(true_risk_index)

# Add a single weight column
df["weight"] = 1.0

# Save to parquet with error handling
output_path = "../data/raw/segmentation_data.parquet"
try:
    # Try with pyarrow engine first
    df.to_parquet(output_path, engine='pyarrow')
    print(f"Synthetic data generated and saved to {output_path} (using pyarrow)")
except Exception as e:
    print(f"PyArrow failed with error: {e}")
    try:
        # Fallback to fastparquet engine
        df.to_parquet(output_path, engine='fastparquet')
        print(f"Synthetic data generated and saved to {output_path} (using fastparquet)")
    except Exception as e2:
        print(f"FastParquet also failed: {e2}")
        # Final fallback to pickle
        output_path_pickle = "../data/raw/segmentation_data.pkl"
        df.to_pickle(output_path_pickle)
        print(f"Saved as pickle instead: {output_path_pickle}")
        # Update output_path for consistency
        output_path = output_path_pickle

df.head()

Synthetic data generated and saved to ../data/raw/segmentation_data.parquet (using pyarrow)


,age,income,credit_score,market_premium,region,prediction_A,target_A,prediction_B,target_B,prediction_C,target_C,weight
0,56,31626.419807,693,0.295256,West,0.097585,0,0.091178,0,0.087754,0,1.0
1,69,94350.676790,565,0.230289,East,0.393303,0,0.346978,1,0.317891,0,1.0
2,46,34226.773664,803,0.112133,South,0.051926,0,0.050744,0,0.050761,0,1.0
3,32,189107.488476,433,0.410269,North,0.220123,0,0.221539,0,0.226736,1,1.0
4,60,32588.089714,325,0.305161,South,0.230528,0,0.207231,0,0.192646,0,1.0
